---
layout: post
title: N-Body
---

In [1]:
from pathlib import Path
import os
import functools

from IPython.display import HTML, Image
import matplotlib.pyplot as plt
from numpy import *
from celluloid import Camera
import matplotlib.patches as patches
from scipy.integrate import odeint, solve_ivp
from simple_pid import PID

ROOT = Path("./assets/img/")

if not os.path.exists(ROOT):
    os.makedirs(ROOT)

N-Body problem.

$$
\frac{d^2 r}{dt^2} = -G \sum_{i=1}^N \frac{m_i}{|r_i|^2} r_i
$$

In [35]:
G = 1e-3
BODY_NUM = 6
MASS = [1e4] + [1]*(BODY_NUM-1)

In [36]:
def get_bodies():

    R = 1
    V = 1

    state_arr = [[0., 0., 0., 0., 0., 0.]]

    theta_arr = linspace(0, 2*pi, BODY_NUM)[:-1]
    
    for theta in theta_arr:
        x = R*cos(theta)
        y = R*sin(theta)
        z = 0

        vx = -V*sin(theta)
        vy = V*cos(theta)
        vz = 0

        state_arr.append([x, y, z, vx, vy, vz])
        
    return array(state_arr)

In [37]:
def motion_step(s0, t):

    s0 = s0.reshape(-1, 6)
    N = s0.shape[0]

    s = []
    for i in range(N):
        
        ri0, vi0 = s0[i,:3], s0[i,3:]        
        dr = vi0
        dv = zeros(3)
        for j in range(N):
            if i != j:
                
                rj0 = s0[j,:3]
                rj = ri0 - rj0
                
                dv += -G * MASS[j] * rj / (linalg.norm(rj)**3+1e-7)
        s.append(concat([dr, dv], axis=0))

    s = array(s).reshape(-1)
    return s

In [39]:
def sim_n_body():

    rng = random.default_rng(seed=0)

    T = 20
    t = linspace(0, T, 500)

    s0 = get_bodies()
    
    sol = solve_ivp(lambda t, s: motion_step(s, t), (0, T), s0.reshape(-1), t_eval=t)

    sol = sol.y.transpose(1, 0)
    
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')

    camera = Camera(fig)
    
    for step in sol:
        step = step.reshape(-1, 6)

        r = step[:,:3]
        ax.scatter(r[:,0], r[:,1], r[:,2], color="black")
        camera.snap()

    anim = camera.animate()
    plt.close()

    gif_path = ROOT / "nbody.gif"  
    anim.save(gif_path, writer="pillow", fps=10)
    return Image(url=gif_path)

sim_n_body()